In [1]:
from vietadminunits import parse_address
import pandas as pd
import re
from unidecode import unidecode

In [0]:
def create_key(text):
    if not isinstance(text, str):
        return text
    u_text = unidecode(text) # Unidecode first help remove special characters
    c_text = u_text.lower().strip() # Case must be second step
    c_text = re.sub(r"\-|\'", ' ', c_text)
    c_text = c_text.replace(' ', '')
    return c_text

In [ ]:
def find_new_district_english(text):
    au = parse_address(text, 2)
    return au.district_english

In [ ]:
def prepare_address(province, change):
    address = province + ' ' + re.sub(f'.* thành ', '', change)
    return address

In [ ]:
def create_province_english(text):
    au = parse_address(text, 1)
    return au.province_english

In [ ]:
df = pd.read_clipboard()

In [ ]:
df

In [ ]:
df['wrong_province'] = df.ward.apply(create_province_english)

In [ ]:
df['true_province'] = df.province.apply(create_province_english)

In [ ]:
df['wrong_province_key'] = df['wrong_province'].apply(create_key)

In [ ]:
df['true_province_key'] = df['true_province'].apply(create_key)

In [ ]:
data = {}

In [ ]:
for row in df.itertuples():
    data[row.wrong_province_key] = df[df['wrong_province_key']==row.wrong_province_key]['true_province_key'].tolist()

In [ ]:
data

In [2]:
df = pd.read_clipboard()

In [3]:
df['full_address'] = df['ward'] + ', ' + df['district'] + ', ' + df['province']

In [4]:
df['district_address'] = df['district'] + ', ' + df['province']

In [5]:
def create_district_key(text):
    au = parse_address(text)
    return au.district_key

In [6]:
df['true_district_key'] = df['district_address'].apply(create_district_key)

In [7]:
df['wrong_district_key'] = df['full_address'].apply(create_district_key)

In [8]:
df

,ward,district,province,full_address,district_address,true_district_key,wrong_district_key
0,Phường Trung Hoà,Quận Cầu Giấy,Thành phố Hà Nội,"Phường Trung Hoà, Quận Cầu Giấy, Thành phố Hà Nội","Quận Cầu Giấy, Thành phố Hà Nội",caugiay,unghoa
1,Phường Thanh Trì,Quận Hoàng Mai,Thành phố Hà Nội,"Phường Thanh Trì, Quận Hoàng Mai, Thành phố Hà...","Quận Hoàng Mai, Thành phố Hà Nội",hoangmai,thanhtri
2,Xã Thanh Xuân,Huyện Sóc Sơn,Thành phố Hà Nội,"Xã Thanh Xuân, Huyện Sóc Sơn, Thành phố Hà Nội","Huyện Sóc Sơn, Thành phố Hà Nội",socson,thanhxuan
3,Xã Tả Thanh Oai,Huyện Thanh Trì,Thành phố Hà Nội,"Xã Tả Thanh Oai, Huyện Thanh Trì, Thành phố Hà...","Huyện Thanh Trì, Thành phố Hà Nội",thanhtri,thanhoai
4,Xã Trung Hòa,Huyện Chương Mỹ,Thành phố Hà Nội,"Xã Trung Hòa, Huyện Chương Mỹ, Thành phố Hà Nội","Huyện Chương Mỹ, Thành phố Hà Nội",chuongmy,unghoa
5,Xã Mường Lang,Huyện Phù Yên,Tỉnh Sơn La,"Xã Mường Lang, Huyện Phù Yên, Tỉnh Sơn La","Huyện Phù Yên, Tỉnh Sơn La",phuyen,muongla
6,Xã Mường Lầm,Huyện Sông Mã,Tỉnh Sơn La,"Xã Mường Lầm, Huyện Sông Mã, Tỉnh Sơn La","Huyện Sông Mã, Tỉnh Sơn La",songma,muongla
7,Xã Mường Lạn,Huyện Sốp Cộp,Tỉnh Sơn La,"Xã Mường Lạn, Huyện Sốp Cộp, Tỉnh Sơn La","Huyện Sốp Cộp, Tỉnh Sơn La",sopcop,muongla
8,Thị trấn Yên Thế,Huyện Lục Yên,Tỉnh Yên Bái,"Thị trấn Yên Thế, Huyện Lục Yên, Tỉnh Yên Bái","Huyện Lục Yên, Tỉnh Yên Bái",lucyen,tranyen
9,Thị trấn Yên Bình,Huyện Yên Bình,Tỉnh Yên Bái,"Thị trấn Yên Bình, Huyện Yên Bình, Tỉnh Yên Bái","Huyện Yên Bình, Tỉnh Yên Bái",yenbinh,tranyen


In [12]:
data = {}
for row in df.itertuples():
    data[row.wrong_district_key] = df[df['wrong_district_key']==row.wrong_district_key]['true_district_key'].tolist()

In [9]:
df[df.true_district_key==df.wrong_district_key]

,ward,district,province,full_address,district_address,true_district_key,wrong_district_key


In [13]:
data

{'unghoa': ['caugiay', 'chuongmy'],
 'thanhtri': ['hoangmai'],
 'thanhxuan': ['socson'],
 'thanhoai': ['thanhtri'],
 'muongla': ['phuyen', 'songma', 'sopcop'],
 'tranyen': ['lucyen', 'yenbinh'],
 'uongbi': ['mongcai'],
 'halong': ['vandon'],
 'haiduong': ['binhgiang'],
 'phucu': ['hungyen'],
 'dongson': ['thanhhoa', 'bimson'],
 'thanhhoa': ['nhuxuan'],
 'kyson': ['tanky'],
 'vinh': ['anhson', 'yenthanh', 'hoangmai'],
 'huongthuy': ['hue', 'hue', 'hue'],
 'giang': ['thangbinh', 'bactramy', 'nuithanh'],
 'tramy': ['bactramy'],
 'sontinh': ['sontay'],
 'ducpho': ['moduc'],
 'iapa': ['chuse'],
 'krongbuk': ['krongpac'],
 'hoathanh': ['chauthanh'],
 'thongnhat': ['bienhoa'],
 'tanphu': ['thuduc', 'quan7'],
 'tanan': ['canduoc'],
 'tanphuoc': ['gocongdong'],
 'cainhum': ['mangthit'],
 'chauphu': ['chaudoc', 'chaudoc'],
 'anminh': ['chauthanh', 'uminhthuong']}